In [ ]:
!pip install transformers
!pip install torch

In [ ]:
import os
print(os.getcwd())

# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive
drive.mount('./MyDrive')

In [ ]:
cd MyDrive/MyDrive/Capstone/code_data

---

In [2]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
raw_text = pd.read_csv('./data/raw_text_tokens.csv')
y = pd.read_csv('./data/y_basic.csv')
x = pd.read_csv('./data/X_basic.csv')
notembedded = pd.read_csv('./data/data_notembeded.csv')

In [11]:
data = pd.concat([raw_text.text, y], axis=1).reset_index(drop=True)

In [12]:
# Load test data
test_data = pd.read_csv('data/data_valid_notembeded.csv')

# Keep important columns
test_data = test_data[['text', 'isRumor']]

# Display 5 samples from the test data
test_data.sample(5)

,text,isRumor
47,@KunstmuseumBern declares that it will accept ...,1
109,#CorneliusGurlitt cousin lodges direct claim f...,0
364,Moscow police investigating #Putin's #Nemtsov ...,0
280,Where are you #Putin? Are you dead❓😐 *awkward ...,0
304,After #Putin disappeared Russian TV no longer ...,0


In [13]:
from sklearn.model_selection import train_test_split

X = data.text.values
y = data.isRumor.values

X_train, X_val, y_train, y_val =\
    train_test_split(X, y, test_size=0.1, random_state=2020)

In [14]:
from sklearn.metrics import accuracy_score, roc_curve, auc

def evaluate_roc(probs, y_true):
    """
    - Print AUC and accuracy on the test set
    - Plot ROC
    @params    probs (np.array): an array of predicted probabilities with shape (len(y_true), 2)
    @params    y_true (np.array): an array of the true values with shape (len(y_true),)
    """
    preds = probs[:, 1]
    fpr, tpr, threshold = roc_curve(y_true, preds)
    roc_auc = auc(fpr, tpr)
    print(f'AUC: {roc_auc:.4f}')
       
    # Get accuracy over the test set
    y_pred = np.where(preds >= 0.5, 1, 0)
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {accuracy*100:.2f}%')
    
    # Plot ROC AUC
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [15]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.
